In [1]:
import torch
import numpy as np
import gzip 
import pickle


In [2]:
from utils import Data

In [3]:
# filename_tagnn_input_data = '..\YooChoose 1_64 Dataset Preparation\ziob_yoochoose_1_64n.pcklz'
filename_tagnn_input_data = '../YooChoose 1_64 Dataset Preparation/ziob_yoochoose_1_64n.pcklz'

batch_size = 32
number_of_nodes = -1

with gzip.open(filename_tagnn_input_data, 'rb') as f:
    train_data = pickle.load(f)
    test_data = pickle.load(f)
    pids = pickle.load(f)

if number_of_nodes == -1:
    number_of_nodes = len(pids)

sessions = test_data[0]
train_data = Data(train_data, shuffle=True)
slices = train_data.generate_batch(batch_size)

In [4]:
seen_edges = set()
c = 0
p1_list = []
p2_list = []
for i, j in zip(slices, np.arange(len(slices))):
    if c == 10:
        break
    alias_inputs, A, items, mask, targets = train_data.get_slice(i)

    alias_inputs = torch.Tensor(alias_inputs).long()
    items = torch.Tensor(items).long()
    A = torch.Tensor(np.array(A)).float()
    mask = torch.Tensor(mask).long()
    sessions = torch.gather(items, 1, alias_inputs).squeeze(0)
    for j, s in enumerate(sessions):
        session = s[:mask[j].sum()]
        for i in range(len(session) - 1):
            p1 = session[i].item()  
            p2 = session[i + 1].item()
            if (p1, p2) not in seen_edges:
                p1_list.append(p1)
                p2_list.append(p2)
                seen_edges.add((p1, p2))  

    

    
p1_tensor = torch.tensor(p1_list)
p2_tensor = torch.tensor(p2_list)
    

In [5]:
edge_index = torch.stack([p1_tensor, p2_tensor])

In [6]:
edge_index

tensor([[13761, 13761, 13758,  ...,  1452, 13792,  4395],
        [13761, 13758, 13758,  ...,  1451, 10684,  5586]])

In [7]:
torch.save(edge_index, 'edge_index.pt')
